In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import gym
import pathlib
import typing as tp
from scipy.special import softmax
import pandas as pd
import os

In [5]:
sys.path.insert(0, '/mnt/ubuntu_data_hdd/school/masters/uwaterloo/1b/cs885/project/codenames/codenames_ai/src')

In [6]:
from default_game import *
from codenames import *
from codenames_env import *

In [7]:
import stable_baselines3
from stable_baselines3.common import env_checker
from stable_baselines3 import DQN, SAC, PPO, A2C, HerReplayBuffer
import matplotlib.pyplot as plt

from stable_baselines3 import TD3
from stable_baselines3.common import results_plotter
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback, CallbackList

In [8]:
from stable_baselines3.common import env_checker

In [10]:
env = CodenamesEnvHack(glove, wordlist)

In [12]:
??env

Type:        CodenamesEnvHack
String form: <CodenamesEnvHack instance>
File:        /mnt/ubuntu_data_hdd/school/masters/uwaterloo/1b/cs885/project/codenames/codenames_ai/src/codenames_env.py
Source:     
class CodenamesEnvHack(CodenamesEnv):
    """Codenames environment for gym.

    Hack the action and observation spaces to make more agents work with it.
    """

    environment_name = "Codenames v0.0.1"
    metadata = {"render.modes": ["human"]}

    def __init__(
        self, glove: cn.Glove, wordlist: cn.WordList, seed: tp.Optional[int] = None
    ):
        super().__init__(glove, wordlist, seed)
        self.action_space = hacked_action_space()
        self.observation_space = spaces.Dict(
            {
                "observation": hacked_observation_space(),
                "desired_goal": hacked_goal_space(),
                "achieved_goal": hacked_goal_space(),
            }
        )

    def current_observation(self):
        return encode_observation(super().current_obse

In [109]:
board_labels = env.current_observation()[25:29]

0.0

In [40]:
our_team_labels = env.current_observation()[25]

In [46]:
our_team_indices = np.where(our_team_labels==1)[0]

In [94]:
def get_our_labels(env):
    our_team_labels = env.current_observation()[25]
    our_team_indices = np.where(our_team_labels==1)[0]
    return our_team_indices

In [140]:
def has_been_guessed(env, index):
    return env.current_observation()[30, index]

In [54]:
one_hot_encode(np.array([2]), 4)

array([[0., 0., 1., 0.]], dtype=float32)

In [79]:
def make_hacked_action(word_index: int, candidate_index: int):
    words = one_hot_encode(np.array([word_index]), NUM_WORDS)
    word_action = np.vstack([1-words, words]).T
    candidate_action = one_hot_encode(np.array([candidate_index]), NUM_HINT_CANDIDATES).reshape(-1, 2)
    return np.vstack([word_action, candidate_action]).flatten()

In [88]:
for i in range(NUM_WORDS):
    for j in range(NUM_HINT_CANDIDATES):
        wtc, hc = decode_action(make_hacked_action(i, j))
        wi = np.where(wtc == 1)[0]
        assert wi.shape[0] == 1
        assert wi[0] == i
        assert hc == j

In [143]:
def execute_trivial_strategy(candidate_index:int = 1):
    env = CodenamesEnvHack(glove, wordlist)
    env.reset()
    env.render()
    done = False
    tot_reward = 0
    our_labels = get_our_labels(env)
    label_index = 0
    while not done:
        l = our_labels[label_index]
        action = make_hacked_action(l, candidate_index)
        print(l, env.board.words[l])
        if done:
            print("Error: game should not be done")
            env.render()
            return tot_reward, False
        obs, reward, done, debug = env.step(action)
        tot_reward += reward
        if has_been_guessed(env, l):
            label_index += 1
        env.render()
        print("Reward", reward)
        print()
    if not done:
        print("Error: Game should be done.")
        return tot_reward, False
    elif reward == -25:
        print("Game lost.")
        return tot_reward, False
    else:
        print("Game won.")
        return tot_reward, True

In [120]:
env.board.words

array(['NEEDLE', 'EMBASSY', 'RACKET', 'BUG', 'BELL', 'BACK', 'WORM',
       'HOSPITAL', 'HEAD', 'POST', 'PASS', 'BAT', 'MERCURY', 'BERRY',
       'SOCK', 'EGYPT', 'AMBULANCE', 'FOREST', 'FRANCE', 'CHEST',
       'BUTTON', 'COMIC', 'PUMPKIN', 'PANTS', 'EAGLE'], dtype='<U11')

In [124]:
env.generate_candidates(["PUMPKIN"], 3)

array(['PIE', 'POTATO', 'PIES', 'PIE', 'POTATO', 'PIES'], dtype='<U68')

In [144]:
trivial_results = []
for i in range(10):
    print("Game", i)
    trivial_results.append(execute_trivial_strategy())
    print("-" * 24)

Game 0
[['PLANE_BY' 'CARD_RE' 'DUCK_BL' 'BOOT_RE' 'CROWN_AS']
 ['VET_RE' 'FORK_BL' 'SNOWMAN_BY' 'LEMON_RE' 'JAM_RE']
 ['MOUNT_BL' 'KNIFE_BY' 'WEB_RE' 'WIND_BY' 'AMAZON_BY']
 ['MILLIONAIRE_BL' 'TRUNK_BL' 'NURSE_BL' 'DWARF_BY' 'MINE_BL']
 ['ANGEL_BL' 'TICK_BY' 'RABBIT_RE' 'KID_RE' 'FIGURE_BL']]
It is BLUE's turn.
The bot is on the BLUE team.
Remaining words: {'ASSASSIN': {'CROWN'}, 'BLUE': {'NURSE', 'DUCK', 'MOUNT', 'FORK', 'TRUNK', 'ANGEL', 'MINE', 'FIGURE', 'MILLIONAIRE'}, 'BYSTANDER': {'KNIFE', 'AMAZON', 'DWARF', 'WIND', 'SNOWMAN', 'TICK', 'PLANE'}, 'RED': {'WEB', 'JAM', 'VET', 'CARD', 'BOOT', 'RABBIT', 'LEMON', 'KID'}}
Remaining word count: {'ASSASSIN': 1, 'BLUE': 9, 'BYSTANDER': 7, 'RED': 8}
Hint history: []
Guessed words []

2 DUCK
[['PLANE_BY' 'CARD_RE' 'duck_bl' 'BOOT_RE' 'CROWN_AS']
 ['VET_RE' 'FORK_BL' 'SNOWMAN_BY' 'LEMON_RE' 'JAM_RE']
 ['MOUNT_BL' 'KNIFE_BY' 'WEB_RE' 'WIND_BY' 'AMAZON_BY']
 ['MILLIONAIRE_BL' 'TRUNK_BL' 'NURSE_BL' 'DWARF_BY' 'MINE_BL']
 ['ANGEL_BL' 'TICK_BY' 'R

In [145]:
trivial_results

[(-8, True),
 (-10, True),
 (-7, True),
 (-8, True),
 (-8, True),
 (-8, True),
 (-9, True),
 (-9, True),
 (-9, True),
 (-28, False)]

In [147]:
results = np.array([i[0] for i in trivial_results])
print(results.mean(), results.std())

-10.4 5.9194594347794975


In [29]:
env.render()

[['NEEDLE_RE' 'EMBASSY_BY' 'RACKET_RE' 'BUG_RE' 'BELL_BY']
 ['BACK_BY' 'WORM_RE' 'HOSPITAL_BL' 'HEAD_RE' 'POST_BL']
 ['PASS_BL' 'BAT_AS' 'MERCURY_BY' 'BERRY_BL' 'SOCK_BL']
 ['EGYPT_BL' 'AMBULANCE_BY' 'FOREST_BL' 'FRANCE_BL' 'CHEST_RE']
 ['BUTTON_RE' 'COMIC_BY' 'PUMPKIN_BY' 'PANTS_BL' 'EAGLE_RE']]
It is BLUE's turn.
The bot is on the BLUE team.
Remaining words: {'ASSASSIN': {'BAT'}, 'BLUE': {'EGYPT', 'POST', 'HOSPITAL', 'FOREST', 'FRANCE', 'SOCK', 'PANTS', 'PASS', 'BERRY'}, 'BYSTANDER': {'AMBULANCE', 'EMBASSY', 'PUMPKIN', 'BELL', 'COMIC', 'MERCURY', 'BACK'}, 'RED': {'BUG', 'CHEST', 'WORM', 'EAGLE', 'RACKET', 'NEEDLE', 'BUTTON', 'HEAD'}}
Remaining word count: {'ASSASSIN': 1, 'BLUE': 9, 'BYSTANDER': 7, 'RED': 8}
Hint history: []
Guessed words []

